In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time
import os

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [19]:
data_dir = r"D:\python all collection\kopyae_2\crack_detect\canny_op"  # Replace with your dataset path
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "valid")
test_dir = os.path.join(data_dir, "test")

In [20]:
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_classes = 2  # Cracked and uncracked screens

In [21]:
transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        # transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


In [22]:
train_dataset = datasets.ImageFolder(train_dir, transform=transform["train"])
val_dataset = datasets.ImageFolder(val_dir, transform=transform["val"])
test_dataset = datasets.ImageFolder(test_dir, transform=transform["test"])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
# Load MobileNetV2
model = models.mobilenet_v2(pretrained=True)


In [24]:
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

In [25]:
model = model.to(device)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    start_time = time.time()
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        # Training phase
        model.train()
        train_loss, correct_train = 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            correct_train += (outputs.argmax(1) == labels).sum().item()

        train_loss /= len(train_loader.dataset)
        train_acc = correct_train / len(train_loader.dataset)

        # Validation phase
        model.eval()
        val_loss, correct_val = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                correct_val += (outputs.argmax(1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = correct_val / len(val_loader.dataset)

        print(f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    end_time = time.time()
    print(f"Training completed in {(end_time - start_time) / 60:.2f} minutes.")

    return model


In [32]:
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)
torch.save(model.state_dict(), "mobilenetv2_crack_detection_3.pth")
print("Model saved!")

Epoch 1/10
----------
Train Loss: 0.0002 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 2/10
----------
Train Loss: 0.0001 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 3/10
----------
Train Loss: 0.0001 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 4/10
----------
Train Loss: 0.0003 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 5/10
----------
Train Loss: 0.0001 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 6/10
----------
Train Loss: 0.0009 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 7/10
----------
Train Loss: 0.0006 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 8/10
----------
Train Loss: 0.0004 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 9/10
----------
Train Loss: 0.0001 Acc: 1.0000
Val Loss: 0.0000 Acc: 1.0000
Epoch 10/10
----------
Train Loss: 0.0065 Acc: 0.9960
Val Loss: 0.0063 Acc: 1.0000
Training completed in 0.40 minutes.
Model saved!


: 

In [29]:
# Test the model
def test_model(model, test_loader):
    model.eval()
    correct_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            correct_test += (outputs.argmax(1) == labels).sum().item()

    test_acc = correct_test / len(test_loader.dataset)
    print(f"Test Accuracy: {test_acc:.4f}")

In [30]:
# Load the model for testing
model.load_state_dict(torch.load("mobilenetv2_crack_detection.pth"))
test_model(model, test_loader)

C:\Users\KoPya\AppData\Local\Temp\ipykernel_5440\1987885992.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mobilenetv2_crack_detection

Test Accuracy: 1.0000


In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np

# Function to evaluate metrics
def evaluate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Compute metrics
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))
    print("\nConfusion Matrix:")
    print(conf_matrix)

    return precision, recall, f1, conf_matrix

# Evaluate the model on the test set
precision, recall, f1, conf_matrix = evaluate_metrics(model, test_loader)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")



Classification Report:
              precision    recall  f1-score   support

     cracked       1.00      1.00      1.00        11
      normal       1.00      1.00      1.00        10

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21


Confusion Matrix:
[[11  0]
 [ 0 10]]
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
